In [6]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import torch
from torchvision import models, transforms
from torch import nn
import pytorch_lightning as pl
import json
import os

class CarClassifier(pl.LightningModule):
    def __init__(self, num_classes=196, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        self.model = models.efficientnet_v2_s(weights=None)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    def forward(self, x): return self.model(x)

class CarClassifierConvNext(pl.LightningModule):
    def __init__(self, num_classes=196, learning_rate=1e-4):
        super().__init__()
        self.save_hyperparameters()
        self.model = models.convnext_tiny(weights=None)
        in_features = self.model.classifier[2].in_features
        self.model.classifier[2] = nn.Linear(in_features, num_classes)
    def forward(self, x): return self.model(x)

class CarApp:
    def __init__(self, root):
        self.root = root
        self.root.title("System Rozpoznawania Pojazdów")
        self.root.geometry("600x750")
        self.root.resizable(False, False)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.load_resources()

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        # --- INTERFEJS GRAFICZNY (GUI) ---

        # 1. Tytuł
        title_label = tk.Label(root, text="Analiza Modelu Samochodu", font=("Helvetica", 18, "bold"), pady=10)
        title_label.pack()

        # 2. Miejsce na obrazek
        self.image_label = tk.Label(root, text="Brak zdjęcia", bg="#eeeeee", width=50, height=20)
        self.image_label.pack(pady=10)

        # 3. Przycisk
        btn = tk.Button(root, text="📂 Wybierz zdjęcie z dysku", command=self.upload_image,
                        font=("Helvetica", 12), bg="#007acc", fg="white", padx=20, pady=10)
        btn.pack(pady=10)

        # 4. Wyniki - EfficientNet
        frame_eff = tk.Frame(root, pady=10)
        frame_eff.pack(fill="x", padx=20)
        tk.Label(frame_eff, text="EfficientNet V2:", font=("Helvetica", 12, "bold"), fg="blue").pack(anchor="w")
        self.result_eff = tk.Label(frame_eff, text="Oczekiwanie na zdjęcie...", font=("Helvetica", 11), justify="left")
        self.result_eff.pack(anchor="w")

        # 5. Wyniki - ConvNeXt
        frame_conv = tk.Frame(root, pady=10)
        frame_conv.pack(fill="x", padx=20)
        tk.Label(frame_conv, text="ConvNeXt Tiny:", font=("Helvetica", 12, "bold"), fg="red").pack(anchor="w")
        self.result_conv = tk.Label(frame_conv, text="Oczekiwanie na zdjęcie...", font=("Helvetica", 11), justify="left")
        self.result_conv.pack(anchor="w")

    def load_resources(self):
        try:
            with open('classes.json', 'r') as f:
                self.class_names = json.load(f)
        except:
            self.class_names = [f"Class {i}" for i in range(196)]
            messagebox.showwarning("Uwaga", "Brak pliku classes.json! Wyniki będą tylko numerami.")

        self.model_eff = self.load_single_model(CarClassifier, "best_model.ckpt")
        self.model_conv = self.load_single_model(CarClassifierConvNext, "best_convnext.ckpt")

    def load_single_model(self, model_class, path):
        if os.path.exists(path):
            try:
                model = model_class.load_from_checkpoint(path, map_location=self.device, num_classes=len(self.class_names))
                model.eval().to(self.device)
                print(f"✅ Załadowano {path}")
                return model
            except Exception as e:
                print(f"❌ Błąd ładowania {path}: {e}")
                return None
        return None

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp")])
        if not file_path:
            return

        try:
            img = Image.open(file_path).convert('RGB')

            img_preview = img.copy()
            img_preview.thumbnail((400, 300))
            img_tk = ImageTk.PhotoImage(img_preview)

            self.image_label.configure(image=img_tk, text="", width=400, height=300)
            self.image_label.image = img_tk

            self.predict(img)

        except Exception as e:
            messagebox.showerror("Błąd", f"Nie udało się otworzyć zdjęcia.\n{e}")

    def predict(self, img_pil):
        img_tensor = self.transform(img_pil).unsqueeze(0).to(self.device)

        # EfficientNet
        if self.model_eff:
            with torch.no_grad():
                preds = self.model_eff(img_tensor)
                probs = torch.softmax(preds, dim=1)[0]
                top3 = torch.topk(probs, 3)

                text = ""
                for i in range(3):
                    cls = self.class_names[top3.indices[i].item()]
                    conf = top3.values[i].item() * 100
                    text += f"• {cls} ({conf:.1f}%)\n"
                self.result_eff.config(text=text)
        else:
            self.result_eff.config(text="Model nie został załadowany.")

        # ConvNeXt
        if self.model_conv:
            with torch.no_grad():
                preds = self.model_conv(img_tensor)
                probs = torch.softmax(preds, dim=1)[0]
                top3 = torch.topk(probs, 3)

                text = ""
                for i in range(3):
                    cls = self.class_names[top3.indices[i].item()]
                    conf = top3.values[i].item() * 100
                    text += f"• {cls} ({conf:.1f}%)\n"
                self.result_conv.config(text=text)
        else:
            self.result_conv.config(text="Model nie został załadowany.")

if __name__ == "__main__":
    root = tk.Tk()
    app = CarApp(root)
    root.mainloop()

✅ Załadowano best_model.ckpt
✅ Załadowano best_convnext.ckpt
